In [1]:
import pandas as pd
import numpy as np
import os
import random
from copy import deepcopy
import math
from itertools import product
import pickle
import joblib
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
user_df = pd.read_csv('./data/user_spec.csv')
loan_df = pd.read_csv('./data/loan_result.csv')

In [3]:
merged_df=pd.merge(user_df, loan_df, on='application_id')

In [4]:
if not os.path.isdir('./customData'):
    os.mkdir('./customData')

In [5]:
merged_df.to_csv('./customData/merged_user_loan.csv')

실습

In [38]:
a=pd.DataFrame([
    list(range(1,10)),
    list(range(8,17)),
    [0]*9,
    list(range(31, 13, -2))
])
a.columns = [f'c_{i}' for i in a.columns]
a.index = [f'i_{i}' for i in a.index]

for i in range(4):
    a.iloc[i,i*3]=np.NaN

IndexError: iloc cannot enlarge its target object

In [39]:
a.loc[['i_0','i_1'],['c_1','c_2']]=1

In [40]:
a

,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8
i_0,NaN,1,1,4.0,5,6,7.0,8,9
i_1,8.0,1,1,NaN,12,13,14.0,15,16
i_2,0.0,0,0,0.0,0,0,NaN,0,0
i_3,31.0,29,27,25.0,23,21,19.0,17,15


In [41]:
a[(a['c_3']!=0) & (a['c_3']!=1)]

,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8
i_0,NaN,1,1,4.0,5,6,7.0,8,9
i_1,8.0,1,1,NaN,12,13,14.0,15,16
i_3,31.0,29,27,25.0,23,21,19.0,17,15


In [47]:
np.isnan(a.iloc[0,0])

True

In [42]:
null_index_list = (merged_df.isnull().sum()!=0)[merged_df.isnull().sum()!=0].index
(merged_df.isnull().sum())[merged_df.isnull().sum()!=0]

birth_year                               128096
gender                                   128096
credit_score                            1509276
yearly_income                                 6
company_enter_month                      400337
personal_rehabilitation_yn              5888701
personal_rehabilitation_complete_yn    11793977
existing_loan_cnt                       2685709
existing_loan_amt                       3890163
loan_limit                                 7382
loan_rate                                  7382
is_applied                              3257239
dtype: int64

## yearly_income                                 

In [45]:
merged_df[merged_df['yearly_income'].isnull()]['user_id']

5781948    670502
5781949    670502
5781950    670502
5781951    670502
5781952    670502
5781953    670502
Name: user_id, dtype: int64

In [47]:
merged_df[merged_df['yearly_income'].isnull()]['user_id'].index

Int64Index([5781948, 5781949, 5781950, 5781951, 5781952, 5781953], dtype='int64')

In [46]:
merged_df[merged_df['user_id']==670502]['yearly_income']

5478320    0.0
5478321    0.0
5478322    0.0
5478323    0.0
5478324    0.0
5478325    0.0
5478326    0.0
5478327    0.0
5478328    0.0
5478329    0.0
5478330    0.0
5781948    NaN
5781949    NaN
5781950    NaN
5781951    NaN
5781952    NaN
5781953    NaN
Name: yearly_income, dtype: float64

In [49]:
for idx in merged_df[merged_df['yearly_income'].isnull()]['user_id'].index:
    merged_df.loc[idx, 'yearly_income'] = 0

In [68]:
merged_df.to_csv('./customData/merged_user_loan.csv')

In [50]:
null_sum = (merged_df.isnull().sum())[merged_df.isnull().sum()!=0]
null_sum

birth_year                               128096
gender                                   128096
credit_score                            1509276
company_enter_month                      400337
personal_rehabilitation_yn              5888701
personal_rehabilitation_complete_yn    11793977
existing_loan_cnt                       2685709
existing_loan_amt                       3890163
loan_limit                                 7382
loan_rate                                  7382
is_applied                              3257239
dtype: int64

# user_id 기반 null 대체

In [122]:
null_sum = (merged_df.isnull().sum())[merged_df.isnull().sum()!=0]
null_sum

birth_year                              128096
gender                                  128096
credit_score                           1509276
company_enter_month                     400337
personal_rehabilitation_yn             5888701
personal_rehabilitation_complete_yn    5888701
existing_loan_cnt                      2685709
existing_loan_amt                      3890163
loan_limit                                7382
loan_rate                                 7382
is_applied                             3257239
dtype: int64

In [125]:
null_list = ['birth_year', 'gender', 'credit_score', 'company_enter_month', 'personal_rehabilitation_yn',
 'personal_rehabilitation_complete_yn', 'existing_loan_cnt', 'existing_loan_amt', 'loan_limit',
 'loan_rate',]

In [146]:
result =dict()

for null_col in null_list:
    user_set = set(merged_df[merged_df[null_col].isnull()]['user_id'])
    cnt = 0
    for user in tqdm(user_set):        
        tmp = merged_df[merged_df['user_id'] == user][['user_id',null_col]].drop_duplicates()
        if len(tmp) == 2:
            new_value_idx = tmp[tmp[null_col].isnull()==False].index[0]
            new_value = tmp.loc[new_value_idx,null_col]
            indexs = merged_df[(merged_df['user_id'] == user) & (merged_df[null_col].isnull()==True)].index
            merged_df.loc[indexs, null_col] = new_value
        elif len(tmp) > 2:
            cnt+=1
    if cnt:
        result[null_col]=cnt

  8%|██████▏                                                                   | 14469/173269 [30:10<5:31:08,  7.99it/s]


KeyboardInterrupt: 

In [147]:
result

{'credit_score': 1584, 'company_enter_month': 3416}

In [148]:
null_sum = (merged_df.isnull().sum())[merged_df.isnull().sum()!=0]
null_sum

birth_year                              108227
gender                                  108227
credit_score                           1350179
company_enter_month                     340617
personal_rehabilitation_yn             5642126
personal_rehabilitation_complete_yn    5888701
existing_loan_cnt                      2685709
existing_loan_amt                      3890163
loan_limit                                7382
loan_rate                                 7382
is_applied                             3257239
dtype: int64

### 분할작업 합치기

In [2]:
merged_df = pd.read_csv('./customData/merged_user_loan.csv', index_col=0)

In [3]:
columns_backup=merged_df.columns

In [11]:
# merged_df_copy1 = pd.read_csv('./customData/merged_user_loan_copy1.csv', index_col=0)[['application_id','personal_rehabilitation_yn',]]
# merged_df_copy3 = pd.read_csv('./customData/merged_user_loan_copy3.csv', index_col=0)[['application_id','existing_loan_amt', 'loan_limit','loan_rate']]

In [12]:
merged_df.shape, merged_df_copy1.shape, merged_df_copy3.shape, 

((13527250, 23), (13527250, 2), (13527250, 4))

In [12]:
merged_df['personal_rehabilitation_yn'] = pd.read_csv('./customData/merged_user_loan_copy1.csv', index_col=0)['personal_rehabilitation_yn']

In [13]:
merged_df[['existing_loan_amt', 'loan_limit','loan_rate']] = pd.read_csv('./customData/merged_user_loan_copy1.csv', index_col=0)[['existing_loan_amt', 'loan_limit','loan_rate']]

In [14]:
merged_df.to_csv('./customData/merged_ver1.csv') 

In [15]:
null_sum = (merged_df.isnull().sum())[merged_df.isnull().sum()!=0]
null_sum

birth_year                              108227
gender                                  108227
credit_score                           1350179
company_enter_month                     340617
personal_rehabilitation_yn             2948659
personal_rehabilitation_complete_yn    5888701
existing_loan_cnt                      2685709
existing_loan_amt                      3890163
loan_limit                                7382
loan_rate                                 7382
is_applied                             3257239
dtype: int64

In [17]:
user_set = set(merged_df[merged_df['company_enter_month'].isnull()]['user_id'])

In [19]:
for user in tqdm(user_set):
    tmp = merged_df[merged_df['user_id'] == user][['user_id','company_enter_month']].drop_duplicates()
    if len(tmp) > 2:
        break

  0%|                                                                                 | 5/36028 [00:00<08:10, 73.47it/s]


In [22]:
tmp

,user_id,company_enter_month
5343655,262165,202104.0
11647441,262165,NaN
11811495,262165,202110.0


## 추가 보충하기 len(tmp)>2

In [29]:
(merged_df.isnull().sum())[merged_df.isnull().sum()!=0]

birth_year                              108227
gender                                  108227
credit_score                           1350179
company_enter_month                     340617
personal_rehabilitation_yn             2948659
personal_rehabilitation_complete_yn    2954456
existing_loan_cnt                      2685709
existing_loan_amt                      3890163
loan_limit                                7382
loan_rate                                 7382
is_applied                             3257239
dtype: int64

birth day는 없음

In [33]:
tmp = merged_df[merged_df['birth_year'].isnull()]

In [35]:
user_set = set(merged_df[merged_df['birth_year'].isnull()]['user_id'])

In [37]:
for user in tqdm(user_set):        
    tmp = merged_df[merged_df['user_id'] == user][['user_id','birth_year']].drop_duplicates()
    if len(tmp) > 2:
        break

100%|██████████████████████████████████████████████████████████████████████████████| 4954/4954 [00:46<00:00, 106.76it/s]


# 의미로 null 채우기

## personal_rehabilitation_complete_yn

In [24]:
merged_df['personal_rehabilitation_yn'].value_counts()

0.0    10531568
1.0       47023
Name: personal_rehabilitation_yn, dtype: int64

In [25]:
merged_df[merged_df['personal_rehabilitation_yn'] == 0]['personal_rehabilitation_complete_yn'].value_counts()

0.0    7597323
Name: personal_rehabilitation_complete_yn, dtype: int64

In [26]:
merged_df[merged_df['personal_rehabilitation_yn'] == 0]['personal_rehabilitation_complete_yn'].isnull().sum()

2934245

In [27]:
merged_df[merged_df['personal_rehabilitation_yn'] == 1]['personal_rehabilitation_complete_yn'].value_counts()

0.0    34591
1.0     6635
Name: personal_rehabilitation_complete_yn, dtype: int64

In [28]:
merged_df[merged_df['personal_rehabilitation_yn'] == 1]['personal_rehabilitation_complete_yn'].isnull().sum()

5797

In [29]:
indexs = merged_df[(merged_df['personal_rehabilitation_yn'] == 0) & (merged_df['personal_rehabilitation_complete_yn'] != 0)].index

In [30]:
merged_df.loc[indexs, 'personal_rehabilitation_complete_yn'] = 0

In [31]:
merged_df[(merged_df['personal_rehabilitation_yn'] == 0) & (merged_df['personal_rehabilitation_complete_yn'] != 0)]

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied


In [32]:
merged_df.to_csv('./customData/merged_ver1_1.csv') 

In [33]:
null_sum = (merged_df.isnull().sum())[merged_df.isnull().sum()!=0]
null_sum

birth_year                              108227
gender                                  108227
credit_score                           1350179
company_enter_month                     340617
personal_rehabilitation_yn             2948659
personal_rehabilitation_complete_yn    2954456
existing_loan_cnt                      2685709
existing_loan_amt                      3890163
loan_limit                                7382
loan_rate                                 7382
is_applied                             3257239
dtype: int64

### 결과백업

In [23]:
merged_df['personal_rehabilitation_yn'].value_counts()

0.0    10531568
1.0       47023
Name: personal_rehabilitation_yn, dtype: int64

In [72]:
merged_df[merged_df['personal_rehabilitation_yn'] == 0]['personal_rehabilitation_complete_yn'].value_counts()

0.0    1692031
1.0         16
Name: personal_rehabilitation_complete_yn, dtype: int64

In [73]:
merged_df[merged_df['personal_rehabilitation_yn'] == 0]['personal_rehabilitation_complete_yn'].isnull().sum()

5905276

In [74]:
merged_df[merged_df['personal_rehabilitation_yn'] == 1]['personal_rehabilitation_complete_yn'].value_counts()

0.0    34591
1.0     6635
Name: personal_rehabilitation_complete_yn, dtype: int64

In [75]:
merged_df[merged_df['personal_rehabilitation_yn'] == 1]['personal_rehabilitation_complete_yn'].isnull().sum()

0

In [93]:
indexs = merged_df[(merged_df['personal_rehabilitation_yn'] == 0) & (merged_df['personal_rehabilitation_complete_yn'] != 0)].index

In [94]:
merged_df.loc[indexs, 'personal_rehabilitation_complete_yn'] = 0

In [95]:
merged_df[(merged_df['personal_rehabilitation_yn'] == 0) & (merged_df['personal_rehabilitation_complete_yn'] != 0)]

,application_id,user_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,...,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied


In [96]:
merged_df.to_csv('./customData/merged_user_loan.csv')

In [97]:
null_sum = (merged_df.isnull().sum())[merged_df.isnull().sum()!=0]
null_sum

birth_year                              128096
gender                                  128096
credit_score                           1509276
company_enter_month                     400337
personal_rehabilitation_yn             5888701
personal_rehabilitation_complete_yn    5888701
existing_loan_cnt                      2685709
existing_loan_amt                      3890163
loan_limit                                7382
loan_rate                                 7382
is_applied                             3257239
dtype: int64

## 

# 타겟 데이터 분석

타겟 데이터엔 null값이 있는가?

In [16]:
target = loan_df[loan_df['is_applied'].isnull()]

In [17]:
X_train = target.loc[:, target.columns !='is_applied']

In [20]:
X_train.shape

(3257239, 6)

In [21]:
X_train_merged=pd.merge(user_df, X_train, on='application_id')

In [22]:
X_train_merged.shape

(3257239, 22)

In [23]:
X_train_merged.isnull().sum()

application_id                               0
user_id                                      0
birth_year                               36470
gender                                   36470
insert_time                                  0
credit_score                            265464
yearly_income                                6
income_type                                  0
company_enter_month                      96769
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn               15472
personal_rehabilitation_complete_yn    2561745
existing_loan_cnt                       541898
existing_loan_amt                       846023
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                1757
loan_rate    

null값을 채운 merged_df엔 있는가?

In [32]:
merged_df = pd.read_csv('./customData/merged_ver1_1.csv', index_col=0)

In [25]:
X_train= merged_df[merged_df['is_applied'].isnull()].loc[:, merged_df.columns !='is_applied']

In [26]:
X_train.shape

(3257239, 23)

In [27]:
X_train.isnull().sum()

Unnamed: 0                                  0
application_id                              0
user_id                                     0
birth_year                              30960
gender                                  30960
insert_time                                 0
credit_score                           256720
yearly_income                               0
income_type                                 0
company_enter_month                     83681
employment_type                             0
houseown_type                               0
desired_amount                              0
purpose                                     0
personal_rehabilitation_yn              14664
personal_rehabilitation_complete_yn     14664
existing_loan_cnt                      541898
existing_loan_amt                      846023
loanapply_insert_time                       0
bank_id                                     0
product_id                                  0
loan_limit                        

null값이... 많네..

In [4]:
set([1,2]).issubset(set([1,2,3]))

True

In [6]:
set(loan_df['application_id'].unique()).issubset(set(user_df['application_id'].unique()))

False

In [15]:
loan_df.sort_index()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN
...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN


In [12]:
1 in `loan_df['application_id']

True

In [10]:
df=loan_df[loan_df['is_applied'].isnull()]

In [11]:
set(df['application_id'].unique()).issubset(set(user_df['application_id'].unique()))

True